# Smart Home Energy Management 
First attempt at getting a working model. 

In [1]:
using POMDPs
using POMDPModelTools
using Random
using Distributions
using POMDPSimulators
using POMDPPolicies
using MCTS
using SARSOP
using Printf
using CSV 
using Plots 
using DataFrames

rng = Random.GLOBAL_RNG; 
pyplot(); 

HIST_RAND_FILENAME = "rand_results.csv"; 
HIST_MCTS_FILENAME = "mcts_results.csv";

## Global Model Parameters 

First, try to get an extremely basic version of the model running. See SIMPLE MODEL DATA PARAMS for details 

### Real Model Data
Need to write function to read in arrays of all the load/solar, occupancy, etc. from CSV files 

In [2]:
# Load Real Data Model Params 

# Import Data from CSV 
function import_data(fileapth)
    # TODO: Write import function 
    # return D, OCC, TOD, t, ODT, TOU
end


# Battery Params 
SOC_MAX = 13500; # Wh 
C_MAX = 5000; # Wh/hr 
C_RES = 100 # Wh (granularity of space) 

# Thermal Params 
TCMFT_LO = 67 # Lower thermal comfort band temp 
TCMFT_HI = 71 # Upper thermal comfort band temp 

# Building Params 
D_MAX = 5000; # Wh/hr 
D_RES = 100 # Wh (granularity of space) 





100

### Slightly more realistic data

In [3]:
SOC_MAX = 13500 # Wh 
C_RATE_MAX = 2000; # Wh
PENALTY_SOC = -10 

TOD_RESOLUTION = 24; 
TOU_SCHEDULE = [
    0.28, 
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.28,
    0.38,
    0.38,
    0.38,
    0.38,
    0.38,
    0.38,
    0.38,
    0.28,
    0.28
]; 
OCC_SCHEDULE = [
    0.99, 0.99, 0.99, 0.99, 0.95, 0.95, 0.75, 0.75, 0.66, 0.33, 0.2, 
    0.4, 0.3, 0.2, 0.3, 0.35, 0.4, 0.75, 0.8, 0.75, 0.8, 0.95, 0.99, 0.99
]; 
# See Plot 

SP_ADJ_SIZE = 3; 

# Thermal Params
TCMFT_LO = 67; 
TCMFT_HI = 71; 
PENALTY_SP = -10; 

# Building Params
SP_MIN = 60; 
SP_MAX = 80; 
PENALTY_DISCOMF = -15; 
HRLY_PROB_SP_ADJ = 0.05; 


PROB_SP_ADJ = HRLY_PROB_SP_ADJ * TOD_RESOLUTION
D_MAX = 5000; 
LOAD_NOISE = 2; 
LOAD_SCHEDULE = [0.02, 0.02, 0.02, 0.02, 0.02, 0.05, 0.1, 0.1, 0.1, 0.05, 0.07, 0.08, 0.04, 0.02, 0.03, 0.05, 0.1, 0.2, 0.15, 0.05, 0.05, 0.05, 0.03, 0.02]
AVG_DAILY_LOAD = 30000 # Wh 

# Weather Params 
ODT_MAX = 80; 
ODT_MIN = 58; 
ODT_NOISE = 1; 

SIM_DAYS = 1
SIM_DURATION = SIM_DAYS * TOD_RESOLUTION; 

DISCOUNT = 0.99;




In [4]:
LOAD_SCHEDULE = [0.02, 0.01, 0.01, 0.01, 0.01, 0.04, 0.07, 0.06,0.05, 0.01, 0.02, 0.06, 0.03, 0.02, 0.03, 0.05, 0.08, 0.12, 0.15, 0.06, 0.03, 0.02, 0.02, 0.02]

24-element Array{Float64,1}:
 0.02
 0.01
 0.01
 0.01
 0.01
 0.04
 0.07
 0.06
 0.05
 0.01
 0.02
 0.06
 0.03
 0.02
 0.03
 0.05
 0.08
 0.12
 0.15
 0.06
 0.03
 0.02
 0.02
 0.02

In [5]:
sum(LOAD_SCHEDULE)

1.0

### SIMPLE Model Data

In [6]:
# Simple Model Params 

#D_ = [3,5,3,9,8,7,5,4,3,2]; # demand (non-hvac) 
#OCC = [1,0,0,1,1,1,0,0,0,1]; # occupancy 
#TOD = [1,2,3,4,5,1,2,3,4,5]; # time of day 
#t = collect(1:10); # time index 
#ODT = [3,2,3,6,5,4,3,2,4,3]; # outdoor temp 
#TOU = [2,2,3,4,2,2,2,3,4,2]; # time of use rate 

##################################
# SIMPLE PARAMS 

SOC_MAX = 13500; 
PENALTY_SOC = -100
C_RATE_MAX = 2000;  
TOD_RESOLUTION = 5; 
TOU_SCHEDULE = [0.28, 0.28, 0.28, 0.38, 0.38]; # $/Wh 
OCC_SCHEDULE = [0.99, 0.02, 0.02, 0.95, 0.95]
SP_ADJ_SIZE = 3; 

# Thermal Params 
TCMFT_LO = 67; # Lower thermal comfort band temp 
TCMFT_HI = 71; # Upper thermal comfort band temp 
PENALTY_SP = -100


# Building Params 
SP_MIN = 60; 
SP_MAX = 80; 
PENALTY_DISCOMF = -150
HRLY_PROB_SP_ADJ = 0.05
PROB_SP_ADJ = HRLY_PROB_SP_ADJ * TOD_RESOLUTION
D_MAX = 10; 
LOAD_NOISE = 2; 
LOAD_SCHEDULE = [0.1, 0.2, 0.05, 0.4, 0.25]
AVG_DAILY_LOAD = 30000 # Wh 

# Weather Params 
ODT_MAX = 77; 
ODT_MIN = 58; 
ODT_NOISE = 2; 

SIM_DURATION = 10; 

DISCOUNT = 0.99;



# Simple Model Params 

D_ = [3,5,3,9,8,7,5,4,3,2]; # demand (non-hvac) 
OCC = [1,0,0,1,1,1,0,0,0,1]; # occupancy 
TOD = [1,2,3,4,5,1,2,3,4,5]; # time of day 
t = collect(1:10); # time index 
ODT = [3,2,3,6,5,4,3,2,4,3]; # outdoor temp 
TOU = [2,2,3,4,2,2,2,3,4,2]; # time of use rate 

##################################
# SIMPLE PARAMS 

SOC_MAX = 10; 
PENALTY_SOC = -20
C_RATE_MAX = 3;  
TOD_RESOLUTION = 5; 
TOU_SCHEDULE = [2,2,5,5,2]; # $/Wh 
OCC_SCHEDULE = [0.99, 0.02, 0.02, 0.95, 0.95]
SP_ADJ_SIZE = 3; 

# Thermal Params 
TCMFT_LO = 4; # Lower thermal comfort band temp 
TCMFT_HI = 6; # Upper thermal comfort band temp 
PENALTY_SP = -10 


# Building Params 
SP_MIN = 1; 
SP_MAX = 10; 
PENALTY_DISCOMF = -15
HRLY_PROB_SP_ADJ = 0.05
PROB_SP_ADJ = HRLY_PROB_SP_ADJ * TOD_RESOLUTION
D_MAX = 10; 
LOAD_NOISE = 2; 
LOAD_SCHEDULE = [0.1, 0.2, 0.05, 0.4, 0.25]
AVG_DAILY_LOAD = 10 # Wh 

# Weather Params 
ODT_MAX = 9; 
ODT_MIN = 1; 
ODT_NOISE = 0.5; 

SIM_DURATION = 20; 

DISCOUNT = 0.99;



## States
Data container representing the state of the smarthome 

struct SmartHomeState
    d_hv::Int64 # demand of hvac (Wh, rounded to nearest 100Wh) 
    d_::Int64 # demand of all other loads (Wh, rounded to nearest 100Wh)
    soc::Int64 # state of charge of battery (Wh, rounded to nearest 100 Wh) 
    rmt::Int64 # room temperature (degF, to nearest degree) 
    occ::Bool # occupancy status 
    hsp::Int64 # heating setpoint (degF, to nearest degree) 
    csp::Int64 # cooling setpoint (degF, to nearest degree) 
    tod::Int64 # time of day (hr, 0-23) 
    odt::Int64 # outdoor temp (degF, to nearest degree) 
    tou::Int64 # time of use rate ($0.01, to nearest cent) 
    t::Int64 # current time index 
end

In [7]:
struct SmartHomeState
    d_hv::Float64 # demand of hvac (Wh, rounded to nearest 100Wh) 
    d_::Float64 # demand of all other loads (Wh, rounded to nearest 100Wh)
    soc::Float64 # state of charge of battery (Wh, rounded to nearest 100 Wh) 
    rmt::Float64 # room temperature (degF, to nearest degree) 
    occ::Bool # occupancy status 
    hsp::Float64 # heating setpoint (degF, to nearest degree) 
    csp::Float64 # cooling setpoint (degF, to nearest degree) 
    tod::Int64 # time of day (hr, 0-23) 
    odt::Float64 # outdoor temp (degF, to nearest degree) 
    tou::Float64 # time of use rate ($0.01, to nearest cent) 
    t::Int64 # current time index 
end

## Actions
Actions are defined by both the charge rate of tha battery and the change in thermostat setpoint 

In [8]:
struct SmartHomeAction
    c::Float64 # Charge rate of battery (Wh, to nearest 100 Wh) 
    dhsp::Float64 # Change in heat setpoint (degF, to nearest degree) 
    dcsp::Float64 # Change in cool setpoint (degF, to nearest degree) 
end

## MDP 
SmartHome data container is defined. It holds all the information needed to define the MDP tuple (S, A, T, R). 


In [9]:
# the smarthome mdp type 
struct SmartHomeMDP <: MDP{SmartHomeState, SmartHomeAction} 
    # Note that our MDP is parametrized by the state and the action
    # Given Data 
    #D_::Array{Float64}
    #OCC::Array{Bool}
    #TOD::Array{Float64}
    #t::Array{Float64}
    #ODT::Array{Float64}
    #TOU::Array{Float64} 
    
    # Battery Params
    soc_max::Float64 
    c_max::Float64
    penalty_soc::Float64
    
    # Comfort Params 
    tcomf_lo::Float64
    tcomf_hi::Float64
    prob_sp_adj::Float64
    penalty_discomf::Float64 
    
    # Building Params 
    d_max::Float64 
    penalty_sp::Float64 
end

#SmartHomeMDP() = SmartHomeMDP(
#    D_, OCC, TOD, t, ODT, TOU, SOC_MAX, C_RATE_MAX, 
#    PENALTY_SOC, TCMFT_LO, TCMFT_HI, PROB_SP_ADJ, 
#    PENALTY_DISCOMF, D_MAX, PENALTY_SP); 

SmartHomeMDP() = SmartHomeMDP(
    SOC_MAX, C_RATE_MAX, 
    PENALTY_SOC, TCMFT_LO, TCMFT_HI, PROB_SP_ADJ, 
    PENALTY_DISCOMF, D_MAX, PENALTY_SP); 

## Define gen 
Implement the complete generative models for both the SmartHomeMDP and SmartHomePOMDP. 

### State transition model 
- d_hv: f(hsp, csp, odt) 
- d_: probabilistic f(d_(t-1), tod, rand)
- soc: f(soc(t-1), c(t-1)) 
- rmt: f(hsp, csp, odt) 
- occ: probabilistic f(tod, rand) 
- hsp: f(hsp(t-1), dhsp, rand) 
- csp: f(csp(t-1), dcsp, rand) 
- tod: f(tod(t-1))
- odt: f(odt(t-1), tod) 
- tou: f(tod) 

### Observation model 
- occ_d: f(occ, p_tp, p_fp) 
    - p_tp: true pos prob, 
    - p_fp: false pos prob
    
### Reward model
Reward model is composed of the following components: 
- TOU charge demand_: f(tou, d_) 
- TOU charge on demand_hvac: f(tou, d_hv) 
- TOU charge on charge_rate: f(tou, c) 
- Discomfort Penalty: f(occ, rmt) 
- SOC violation Penalty: f(soc)  
- SP violation penalty: f(hsp, csp) 
- Demand Charge: f(D_, D_HV, C) depends on all timesteps for the entire duration 


In [20]:
function update_sps(m::SmartHomeMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    if a.dhsp != 0 || a.dcsp != 0 
        # If action=adjust, deterministically set thermostat 
        hsp = min(max(s.hsp+a.dhsp, SP_MIN), SP_MAX) 
        csp = min(max(s.csp+a.dcsp, SP_MIN), SP_MAX) 
        #return s.hsp + a.dhsp, s.csp + a.dcsp
        return hsp, csp 
    else
        # Else, probabilistic adjustment based on thermal comfort 
        if rand(rng) < m.prob_sp_adj
            # Adjust setpoints 
            setpoints = [round(x) for x in rand(rng, Distributions.Uniform(m.tcomf_lo, m.tcomf_hi),2)] # TODO: Figure out why not working 
            center = (m.tcomf_lo + m.tcomf_hi)/2
            #setpoints = [round(min(max(x, m.tcomf_lo), m.tcomf_hi)) for x in rand(rng, Normal(center,2),2)]
            return minimum(setpoints), maximum(setpoints) 
        else
            # Maintain previous SPs
            return s.hsp, s.csp 
        end
    end
end

function hv_model(hsp, csp, odt, rng) 
    # Fitted regression model based hv_model data analysis. 
    # Model was fit to predict kWh/day 
    # d_hv = a*cdd + b*hdd + c + N(0,RMSE)  
    
    cdd = max(odt-csp,0) # cdd in an hour 
    hdd = max(hsp-odt,0) # hdd in an hour 
    
    a = 2.1898; b = 0.9476; c = 0.5394; RMSE = 6.146; 
    error = rand(rng, Normal(0,RMSE), 1)[1]
    
    d_hv = a*cdd + b*hdd + c + error
    d_hv = round(d_hv*1000/float(TOD_RESOLUTION, digits=-2) # kWh/day -> Wh/hr(to nearest 100Wh) 
    
    return d_hv
end

function hv_model_simple(hsp, csp, odt, rng)
    # Extremely simplified model to work with SIMPLE DATA 
    cdd = max(odt-csp,0) # cdd in an hour 
    hdd = max(hsp-odt,0) # hdd in an hour 
    
    a = 1; b = 0.2; RMSE = 0.05; 
    error = rand(rng, Normal(0,RMSE), 1)[1]
    
    d_hv = a*cdd + b*hdd + error 
    d_hv = max(min(round(d_hv),10), 0) 
    
    return d_hv
end

function update_d_(d_, tod)
    # TODO: fit model dependent on TOD
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # Maybe fit a BN to this data? 
    noise_d_ = float(LOAD_NOISE)
    d_ = float(LOAD_SCHEDULE[tod]* AVG_DAILY_LOAD) #OVERRIDE PREVIOUS VALUE 
    
    d_ += rand(rng, Normal(0,noise_d_), 1)[1] # Probabilistic Change 
    d_ = max(min(round(d_),10), 0) # Clamp to 0-10 int
    #d_ = max(min(round(d_),10000,0) 
    return d_
end

function update_occ(occ, tod)
    # TODO: fit model dependent on TOD 
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # Maybe fit a BN to this data? 
    p_change_occ = 0.3
    p_occ = OCC_SCHEDULE[tod] 
    
    #return rand(rng) < p_change_occ ? !occ : occ
    return rand(rng) < p_occ 
end

function update_odt(tod)
    # Fitted to a sine wave with noise 
    # TODO: need to adjust equation params when converting to realistic data 
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # TODO: fit model dependent on TOD 
    # Maybe fit a BN to this data? 

    noise_odt = float(ODT_NOISE); 
    temp_max = float(ODT_MAX);
    temp_min = float(ODT_MIN);
    tod_per_day = float(TOD_RESOLUTION);
    
    #noise_odt = 1; 
    #temp_max = 7.5; 
    #temp_min = 2.5;
    #tod_per_day = 5; 
    
    amp = (temp_max - temp_min) / 2
    mean = (temp_max + temp_min) / 2
    
    odt = amp*sin((tod+1.75)*pi/(tod_per_day/2)) + mean
    odt += rand(rng, Normal(0,noise_odt), 1)[1] 
    #odt = max(min(round(odt),temp_max), temp_min) # Clamp to 0-10 int 
    
    return odt
end

function update_tou(tod) 
    #TOU_SCHEDULE = [2,2,3,4,2]; 
    # TODO: Change tou_schedule to take as model input rather than hard code 
    tou = TOU_SCHEDULE[tod] 
    return tou 
end


# MDP Generative Model 
function POMDPs.gen(m::SmartHomeMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    # transition model 
    t = s.t + 1 # Deterministic, fixed 
    #tod = rem(s.tod + 1, 24) # Deterministic, fixed 
    tod = rem(s.tod, TOD_RESOLUTION) + 1 # TODO: THIS IS FOR SIMPLE MODEL 
    
    #d_ = m.D_[s.t+1] # Deterministic, fixed (should/could it be probabilistic?)
    #occ = OCC[s.t+1] # Deterministic, fixed (should/could it be probabilistic?) 
    #odt = ODT[s.t+1] # Deterministic, fixed 
    #tou = TOU[s.t+1] # Deterministic, fixed 
    
    d_ = update_d_(s.d_, tod) # Probabilistic
    occ = update_occ(s.occ, tod) # Probabilistic
    odt = update_odt(tod) # Probabilistic
    tou = update_tou(tod) # By Schedule
    
    
    (hsp, csp) = update_sps(m, s, a, rng) 
    #d_hv = hv_model(hsp, csp, odt, rng) # Based on actual fit 
    #d_hv = hv_model_simple(hsp, csp, s.odt, rng) 
    d_hv = hv_model(hsp, csp, s.odt, rng) 
    soc = min(max(s.soc + a.c, 0), SOC_MAX) 
    rmt = min(max(hsp,odt),csp)
    
    sp = SmartHomeState(d_hv, d_, soc, rmt, occ, hsp, csp, tod, odt, tou, t)
    

    # observation model 
    # N/A
    
    # reward model 
    r = -tou * (s.d_ + s.d_hv + soc - s.soc)
    r += (s.rmt > m.tcomf_hi || s.rmt < m.tcomf_lo) ? m.penalty_discomf : 0 
    r += (s.soc > m.soc_max || s.soc < 0) ? m.penalty_soc : 0 
    r += (s.hsp > s.csp) ? m.penalty_sp : 0 # HSP must be less than or equal to CSP 
    
    
    # create and return a NamedTuple 
    return (sp=sp, r=r) # For MDP 
end



LoadError: syntax: missing comma or ) in argument list

### TODO: Convert above code from MDP -> POMDP

In [11]:
# POMDP Generative Model 
function POMDPs.gen(m::SmartHomePOMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    # transition model 
    # should be same as MDP 
    
    # observation model 
    if sp # Opccupied 
        o = rand(rng) < m.p_occ_tp
    else # Not Occupied 
        o = rand(rng) < m.p_occ_fp
    end
    
    
    # reward model 
    
    
    # create and return a NamedTuple 
    return (sp=sp, o=o, r=r) # For POMDP 
end



UndefVarError: UndefVarError: SmartHomePOMDP not defined

In [12]:
sh = SmartHomeMDP(); 

## Step Through Random Policy 

In [13]:
# SIMPLE VALUES 
#POMDPs.initialstate(m::SmartHomeMDP, rng::MersenneTwister) = SmartHomeState(5, 5, 5, 5, true, 4, 6, 1, 5, 2, 1)  
#POMDPs.initialstate_distribution(m::SmartHomeMDP) = SparseCat([SmartHomeState(5, 5, 5, 5, true, 4, 6, 1, 5, 2, 1), SmartHomeState(4, 5, 5, 5, true, 4, 6, 1, 5, 2, 1)], [0.4, 0.6])



# Real Values Initial State
POMDPs.initialstate(m::SmartHomeMDP, rng::MersenneTwister) = SmartHomeState(0, 0, 0.5*SOC_MAX, 68, true, 68, 70, 1, 66, TOU_SCHEDULE[1], 1)  
#POMDPs.initialstate_distribution(m::SmartHomeMDP) = SparseCat([SmartHomeState(5, 5, 5, 5, true, 4, 6, 1, 5, 2, 1), SmartHomeState(4, 5, 5, 5, true, 4, 6, 1, 5, 2, 1)], [0.4, 0.6])


# TODO: Enumerate more actions 
POMDPs.actions(m::SmartHomeMDP) = [
    SmartHomeAction(-C_RATE_MAX ,0,0),            SmartHomeAction(0,0,0),            SmartHomeAction(C_RATE_MAX,0,0), 
    SmartHomeAction(-C_RATE_MAX ,SP_ADJ_SIZE,0),  SmartHomeAction(0,SP_ADJ_SIZE,0),  SmartHomeAction(C_RATE_MAX,SP_ADJ_SIZE,0), 
    SmartHomeAction(-C_RATE_MAX ,0,SP_ADJ_SIZE),  SmartHomeAction(0,0,SP_ADJ_SIZE),  SmartHomeAction(C_RATE_MAX,0,SP_ADJ_SIZE), 
    SmartHomeAction(-C_RATE_MAX ,-SP_ADJ_SIZE,0), SmartHomeAction(0,-SP_ADJ_SIZE,0), SmartHomeAction(C_RATE_MAX,-SP_ADJ_SIZE,0), 
    SmartHomeAction(-C_RATE_MAX ,0,-SP_ADJ_SIZE), SmartHomeAction(0,0,-SP_ADJ_SIZE), SmartHomeAction(C_RATE_MAX,1,-SP_ADJ_SIZE)]

POMDPs.discount(m::SmartHomeMDP) = DISCOUNT



In [14]:
rand_policy = RandomPolicy(sh)
iter = 1 
for (s, a, r) in stepthrough(sh, rand_policy, "s,a,r", max_steps=100)
    if iter < SIM_DURATION
        println(string("TOD: ", s.tod, ", SOC: ", s.soc, ", OCC: ", s.occ, ", ODT: ", s.odt, ", HSP/CSP: ", s.hsp, "/", s.csp)) 
    end
    iter += 1
end

DistributionNotImplemented: 

POMDPs.jl could not find an implementation for DDN Node :sp. Consider the following options:

1) Implement POMDPs.gen(::SmartHomeMDP, ::SmartHomeState, ::SmartHomeAction, ::AbstractRNG) to return a NamedTuple with key :sp.

Closest candidates are:
  gen(::Union{MDP, POMDP}, ::Any, ::Any, ::Any) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/gen_impl.jl:82
  gen(!Matched::GenericDDNNode, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:165
  gen(!Matched::DistributionDDNNode, ::Any, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:117
  ...

2) Implement POMDPs.gen(::DDNNode{:sp}, ::SmartHomeMDP, ::Any, ::Any, ::Any, ::AbstractRNG).

Closest candidates are:
  gen(::DDNNode{x}, ::Any, ::Any...) where x at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/gen_impl.jl:67
  gen(!Matched::GenericDDNNode, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:165
  gen(!Match

## Solve MDP 

Implementing on MonteCarlo Tree Search 

In [15]:
#@requirements_info MCTSSolver() SmartHomeMDP()
n_iter = 100000
depth = TOD_RESOLUTION #* 2
ec = 10.0

solver = MCTSSolver(n_iterations=n_iter,
    depth=depth,
    exploration_constant=ec,
    enable_tree_vis=true
)


policy = solve(solver, sh)
state = initialstate(sh, Random.MersenneTwister(4))

a = action(policy, state)

#using D3Trees
#D3Tree(policy, state, init_expand=2)  # click on the node to expand it

DistributionNotImplemented: 

POMDPs.jl could not find an implementation for DDN Node :sp. Consider the following options:

1) Implement POMDPs.gen(::SmartHomeMDP, ::SmartHomeState, ::SmartHomeAction, ::AbstractRNG) to return a NamedTuple with key :sp.

Closest candidates are:
  gen(::Union{MDP, POMDP}, ::Any, ::Any, ::Any) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/gen_impl.jl:82
  gen(!Matched::GenericDDNNode, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:165
  gen(!Matched::DistributionDDNNode, ::Any, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:117
  ...

2) Implement POMDPs.gen(::DDNNode{:sp}, ::SmartHomeMDP, ::Any, ::Any, ::Any, ::AbstractRNG).

Closest candidates are:
  gen(::DDNNode{x}, ::Any, ::Any...) where x at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/gen_impl.jl:67
  gen(!Matched::GenericDDNNode, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:165
  gen(!Match

## Evaluate Model 

In [16]:
hist_rand = HistoryRecorder(max_steps=SIM_DURATION)
hist_rand = simulate(hist_rand, sh, rand_policy, state)

println("Random Policy Total discounted reward: $(discounted_reward(hist_rand))")

DistributionNotImplemented: 

POMDPs.jl could not find an implementation for DDN Node :sp. Consider the following options:

1) Implement POMDPs.gen(::SmartHomeMDP, ::SmartHomeState, ::SmartHomeAction, ::AbstractRNG) to return a NamedTuple with key :sp.

Closest candidates are:
  gen(::Union{MDP, POMDP}, ::Any, ::Any, ::Any) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/gen_impl.jl:82
  gen(!Matched::GenericDDNNode, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:165
  gen(!Matched::DistributionDDNNode, ::Any, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:117
  ...

2) Implement POMDPs.gen(::DDNNode{:sp}, ::SmartHomeMDP, ::Any, ::Any, ::Any, ::AbstractRNG).

Closest candidates are:
  gen(::DDNNode{x}, ::Any, ::Any...) where x at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/gen_impl.jl:67
  gen(!Matched::GenericDDNNode, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:165
  gen(!Match

In [17]:
hist_mcts = HistoryRecorder(max_steps=SIM_DURATION)
hist_mcts = simulate(hist_mcts, sh, policy, state)

println("Monte Carlo Policy Total discounted reward: $(discounted_reward(hist_mcts))")

DistributionNotImplemented: 

POMDPs.jl could not find an implementation for DDN Node :sp. Consider the following options:

1) Implement POMDPs.gen(::SmartHomeMDP, ::SmartHomeState, ::SmartHomeAction, ::AbstractRNG) to return a NamedTuple with key :sp.

Closest candidates are:
  gen(::Union{MDP, POMDP}, ::Any, ::Any, ::Any) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/gen_impl.jl:82
  gen(!Matched::GenericDDNNode, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:165
  gen(!Matched::DistributionDDNNode, ::Any, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:117
  ...

2) Implement POMDPs.gen(::DDNNode{:sp}, ::SmartHomeMDP, ::Any, ::Any, ::Any, ::AbstractRNG).

Closest candidates are:
  gen(::DDNNode{x}, ::Any, ::Any...) where x at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/gen_impl.jl:67
  gen(!Matched::GenericDDNNode, ::Any...) at /Users/coreyshono/.julia/packages/POMDPs/XDCTo/src/ddn_struct.jl:165
  gen(!Match

## Export Simulation Results 

In [18]:
function export_results(hist, filename)
    # Write to file
    open(filename, "w") do io
        @printf(io, "t,TOD,c,dhsp,dcsp,d_hv,d_,soc,rmt,occ,hsp,csp,odt,tou,r\n")
        for (s, a, r, sp) in eachstep(hist, "(s, a, r, sp)")  
            @printf(io, "%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s\n", s.t, s.tod, a.c, a.dhsp, a.dcsp, s.d_hv, s.d_, s.soc, s.rmt, s.occ, s.hsp, s.csp, s.odt, s.tou, r)
        end
    end
end

export_results (generic function with 1 method)

In [19]:
export_results(hist_rand, HIST_RAND_FILENAME) 
export_results(hist_mcts, HIST_MCTS_FILENAME) 

MethodError: MethodError: no method matching eachstep(::HistoryRecorder, ::String)
Closest candidates are:
  eachstep(!Matched::POMDPSimulators.AbstractSimHistory, ::Any) at /Users/coreyshono/.julia/packages/POMDPSimulators/V9qfg/src/history.jl:126